# Finding axis lengths and other info from sweep objects 

To facilitate real time plotting, it is useful to know the layout of the data to be plotted before the sweep is run. With the layout of the data we mean: 

1. The length of the axes 
2. The min/max value of the axes 
3. The step value of the axes 

It is not always possible to infer these parameters before hand. This notebook explains when it is and isnt possible and how to retrieve this info when available 

In [10]:
import numpy as np
import matplotlib.pyplot as plt

from qcodes import new_experiment
from qcodes.dataset.plotting import plot_by_id
from qcodes.instrument.parameter import ManualParameter
from qcodes.sweep import sweep, setter, run

In [2]:
experiment = new_experiment("sweep_introduction", sample_name="intro")

In [3]:
x = ManualParameter("x")
y = ManualParameter("y")

m = ManualParameter("m")
m.get = lambda:np.sin(x())

n = ManualParameter("n")
n.get = lambda:np.sin(x()**2 + y())

Data layout information is only available if we use the convenience functions (e.g. the function sweep instead of the class Sweep). Furthermore, the second argument of the sweep functions needs to have a well defined length, such as a list or a numpy array

In [4]:
sweep_object = sweep(x, np.linspace(-4, 4, 100))(
    m, 
    sweep(y, np.linspace(-3, 5, 100))(
        n
    )
)

In [5]:
sweep_object.parameter_table.layout_info("m")

{'x': {'min': -4.0, 'max': 4.0, 'length': 100, 'steps': 0.080808080800000001}}

In [6]:
sweep_object.parameter_table.layout_info("n")

{'x': {'min': -4.0, 'max': 4.0, 'length': 100, 'steps': 0.080808080800000001},
 'y': {'min': -3.0, 'max': 5.0, 'length': 100, 'steps': 0.080808080800000001}}

We can only determine min/max and step values if the set points contain numeric values.

In [7]:
x_setpoints = [object() for _ in range(100)]

sweep_object = sweep(x, x_setpoints)(
    m, 
    sweep(y, np.linspace(-3, 5, 100))(
        n
    )
)

print(sweep_object.parameter_table.layout_info("m"))

{'x': {'min': '?', 'max': '?', 'length': 100, 'steps': '?'}}


For step determination we also need to have uniform spacing between the setpoints 

In [8]:
x_setpoints = np.random.normal(-1, 1, 100)

sweep_object = sweep(x, x_setpoints)(
    m, 
    sweep(y, np.linspace(-3, 5, 100))(
        n
    )
)

print(sweep_object.parameter_table.layout_info("m"))

{'x': {'min': -2.9434716184885423, 'max': 1.7304857582877293, 'length': 100, 'steps': '?'}}


Sometimes are cannot derive a length at all 

In [9]:
x_setpoints = (i for i in np.random.normal(-1, 1, 100))

sweep_object = sweep(x, x_setpoints)(
    m, 
    sweep(y, np.linspace(-3, 5, 100))(
        n
    )
)

print(sweep_object.parameter_table.layout_info("m"))

{'x': {'min': '?', 'max': '?', 'length': '?', 'steps': '?'}}


Finally, if our data is dependent on an inferred parameter, it becomes impossible te determine its minimum, maximum and step values. 

In [14]:
@setter([("x", "V")], inferred_parameters=[("x_mv", "mV")])
def mysetter(xv): 
    return xv * 1000

x_setpoints = np.linspace(-4, 4, 100)

sweep_object = sweep(mysetter, x_setpoints)(
    m, 
    sweep(y, np.linspace(-3, 5, 100))(
        n
    )
)

print(sweep_object.parameter_table.layout_info("m"))

{'x_mv': {'min': '?', 'max': '?', 'length': 100, 'steps': '?'}}


In [1]:
import collections

In [2]:
collections.Iterable

collections.abc.Iterable

In [4]:
a = (1, )

isinstance(a, collections.Iterable)

True